# EDA - Titanic (Análise Exploratória)
**Aluno:** Gustavo Silva Lourenço  
**Descrição:** Análise exploratória do dataset Titanic (Kaggle). Objetivos: entender dados, limpar (remover id), tratar valores nulos, visualizar distribuições e correlações, e gerar um resumo com insights.

**Fonte dos dados:** Kaggle - Titanic: Machine Learning from Disaster (arquivo `train.csv`)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6)
sns.set(style='whitegrid')

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
print('Shape:', df.shape)
display(df.info())
display(df.describe(include='all').T)

In [ ]:
df.columns

### Observação sobre a coluna Id
A coluna `PassengerId` não contém informação preditiva útil e pode atrapalhar visualizações, então vamos removê-la depois de inspecionar.

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df = df.drop(columns=['PassengerId'])
df.head()

In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Embarked'].isnull().sum()

In [ ]:
print('Cabin not null:', df['Cabin'].notnull().sum(), '/', len(df))
df['Cabin_letter'] = df['Cabin'].dropna().apply(lambda x: x[0])
df['Cabin_letter'].value_counts()

In [ ]:
df['HasCabin'] = df['Cabin'].notnull().astype(int)
df = df.drop(columns=['Cabin'])
df.head()

In [ ]:
age_median = df.groupby(['Sex','Pclass'])['Age'].median()

def fill_age(row):
    if pd.isnull(row['Age']):
        return age_median.loc[row['Sex'], row['Pclass']]
    else:
        return row['Age']

df['Age'] = df.apply(fill_age, axis=1)
df['Age'].isnull().sum()

In [ ]:
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Fare'].isnull().sum()

In [ ]:
df_encoded = pd.get_dummies(df, columns=['Sex','Embarked','Pclass'], drop_first=True)
df_encoded.columns

In [ ]:
surv_rate = df['Survived'].mean()
print(f'Taxa de sobrevivência: {surv_rate:.2%}')
sns.countplot(x='Survived', data=df)
plt.title('Contagem: Sobreviventes (1) vs Não (0)')
plt.show()

In [ ]:
display(pd.crosstab(df['Sex'], df['Survived'], normalize='index').style.format('{:.2%}'))
sns.countplot(x='Survived', hue='Sex', data=df)
plt.title('Sobrevivência por Sexo')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,5))
sns.histplot(df['Age'], bins=30, ax=axes[0])
axes[0].set_title('Distribuição de Age')
sns.boxplot(y='Age', data=df, ax=axes[1])
axes[1].set_title('Boxplot de Age')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,5))
sns.histplot(df['Fare'], bins=30, ax=axes[0])
axes[0].set_title('Distribuição de Fare (original)')
sns.histplot(np.log1p(df['Fare']), bins=30, ax=axes[1])
axes[1].set_title('Distribuição de log(Fare+1)')
plt.show()

In [ ]:
sns.countplot(x='Pclass', hue='Survived', data=df)
plt.title('Sobrevivência por Pclass')
plt.show()

sns.countplot(x='Embarked', hue='Survived', data=df)
plt.title('Sobrevivência por Porto de Embarque')
plt.show()

In [ ]:
corr = df_encoded.corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Heatmap de Correlação')
plt.show()

In [ ]:
cols_for_pair = ['Survived','Age','Fare','SibSp','Parch']
sns.pairplot(df[cols_for_pair], hue='Survived', diag_kind='hist', corner=True)
plt.suptitle('Pairplot - algumas variáveis numéricas', y=1.02)
plt.show()

In [ ]:
cat_cols = ['Sex','Pclass','Embarked','HasCabin']
for col in cat_cols:
    print('---', col)
    print(df.groupby(col)['Survived'].mean())
    print()

## Conclusões rápidas
- Mulheres e passageiros da 1ª classe tiveram maiores taxas de sobrevivência.
- Idade, tarifa e presença de cabine têm relação com sobrevivência.
- Valores nulos foram tratados: Age (imputado), Cabin (convertido em HasCabin), Embarked (imputado por moda).

In [ ]:
df.to_csv('titanic_cleaned.csv', index=False)
print('Arquivo salvo: titanic_cleaned.csv')